[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1YnDHZY6owEypgSgvJJgXPcntxOjy9GpS?usp=sharing)


# Build a Chatbot with Guardrails: Using Langchain and Portkey to Enforce PII Detection and more

In this tutorial, we'll create two versions of a customer support chatbot:

1. A basic chatbot without PII protection
2. An enhanced chatbot using LangChain and Portkey guardrails to protect sensitive information

By comparing these two implementations, we'll demonstrate the importance and effectiveness of using guardrails to prevent PII exposure in AI applications.


Portkey's **Guardrails** offer real-time enforcement of LLM behavior, with features including:

- **Regex matching** and **JSON Schema validation**
- **Code detection**
- **Custom guardrail**s: Integrate your existing guardrail systems custom guardrail integration
-**LLM-based guardrails** (e.g., gibberish detection, prompt injection scanning)

With **20+ deterministic guardrail**s and integrations with platforms like **Aporia, Pillar, Patronus AI and Prompt Security,** Portkey provides comprehensive AI safety solutions. Guardrails can be configured for inputs, outputs, or both, with actions ranging from request denial to alternative LLM fallbacks.

For more details, visit the [Portkey Guardrails documentation](https://docs.portkey.ai/docs/product/guardrails/list-of-guardrail-checks/patronus-ai).


Let's build our guardrail-protected chatbot!

# Chatbot Without Guardrails

Before we implement PII protection with Portkey guardrails, let's create a basic chatbot using LangChain. This version will not have any PII detection or protection mechanisms.

## Step 1: Setting up the environment

First, let's install the necessary packages:

In [ ]:
!pip install -qU langchain-openai langchain langchain_community

#Step 2: Importing required libraries and setting up API keys

In [ ]:
import os
from typing import Dict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory


model = ChatOpenAI(
    api_key="YOUR_OPENAI_KEY",
    model="gpt-3.5-turbo"
)

In [ ]:
# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
])

# Create a chain that combines the prompt and the model
chain = prompt | model

# Create a dictionary to store chat histories for different sessions
store: Dict[str, BaseChatMessageHistory] = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap the chain with message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

# Function to chat with the bot
def chat_with_bot(session_id: str, user_input: str):
    config = {"configurable": {"session_id": session_id}}
    response1 = with_message_history.invoke(
        {"messages": [{"content": user_input, "type": "human"}]},
        config=config
    )
    return response1.content

Step 4: Running the chatbot

In [ ]:
# Example usage
if __name__ == "__main__":
    session_id = "abc123"

    print("Chatbot Without Guardrails")
    print("Type 'exit', 'quit', or 'bye' to end the conversation.")

    while True:
        user_input1 = input("You: ")
        if user_input1.lower() in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

        response1 = chat_with_bot(session_id, user_input1)
        print(f"Bot: {response1}")

Chatbot Without Guardrails
Type 'exit', 'quit', or 'bye' to end the conversation.
You: hey
Bot: Hello! How can I assist you today?
You: "My name is Siddharth and my email id is xyz@gmail.com
Bot: Hello Siddharth! It's nice to meet you. How can I help you today?
You: what is my email
Bot: Your email is xyz@gmail.com.
You: exit
Bot: Goodbye!


# Build a Chatbot with Guardrails: Using Langchain and Portkey to Enforce PII Detection and more

In this tutorial, we'll create two versions of a customer support chatbot:

1. A basic chatbot without PII protection
2. An enhanced chatbot using LangChain and Portkey guardrails to protect sensitive information

By comparing these two implementations, we'll demonstrate the importance and effectiveness of using guardrails to prevent PII exposure in AI applications.


# Setting Up Portkey Guardrails

Before building our chatbot, let's configure Portkey guardrails for PII detection:

1. Sign Up for [portkey.ai](https://portkey.ai) (the dev account is **free** ferver) 🎊


2. Enable **Patronus AI** in the [Integrations page](https://app.portkey.ai/integrations) on Portkey App


3. Navigate to the [Guardrails dashboard](https://app.portkey.ai/guardrails)


4. Create a new guardrail, by selecting "**Detect PII Guardrail**" by Patronus AI


5. In the Actions tab, select "**Deny the request if guardrail fails**" from the Settings menu.

6. Name your guardrail, create it, and then copy your Config ID for future use.



This **Config ID** is crucial for integrating PII protection into our LangChain-Portkey  chatbot. With this setup, we're ready to create a secure, PII-aware conversational AI.


<img src="https://raw.githubusercontent.com/siddharthsambharia-portkey/Portkey-Product-Images/main/Portkey_PII_Guardrails.png" alt="O" width="600" />

<img src="https://raw.githubusercontent.com/siddharthsambharia-portkey/Portkey-Product-Images/main/actions-gaurdrails.png" alt="O" width="600" />



# Step 1: Setting up the environment

First, let's install the necessary packages:

In [ ]:
!pip install -qU langchain-openai portkey-ai langchain langchain_community

# Step 2: Importing required libraries and setting up API keys

In this step, we'll import the necessary libraries and set up our API keys. We'll also create Portkey config object to add guardrails to protect against PII exposure. Get your Portkey API key from [here](https://app.portkey.ai/api-keys).

In [ ]:
import os
from typing import Dict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from portkey_ai import createHeaders, PORTKEY_GATEWAY_URL
from langchain_core.chat_history import InMemoryChatMessageHistory


portkey_config={
	"after_request_hooks": [
		{
			"id": "pg-portke-416feb" #YOUR_CONFIG_ID
		}
	]
}


llm = ChatOpenAI(
    api_key="YOUR_OPENAI_API_KEY",
    base_url=PORTKEY_GATEWAY_URL,
    model="gpt-3.5-turbo",
    default_headers=createHeaders(
        provider="openai",
        api_key="Your_Portkey_API_key",
        config=portkey_config,
    )
)


This code sets up our environment with the necessary configurations for Portkey guardrails. The portkey_config defines our guardrail settings, including retry attempts, caching, and hooks for request and response processing.



# Step 3: Creating the chat prompt template and chain

Now, let's set up our chat prompt template and create a chain that combines the prompt with our Portkey-protected model. This structure allows us to maintain a conversation history while applying guardrails to each interaction.

In [ ]:

# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability. Reply with the user info given in the request"),
    MessagesPlaceholder(variable_name="messages"),
])

# Create a chain that combines the prompt and the model
chain1 = prompt | llm

# Create a dictionary to store chat histories for different sessions
store: Dict[str, BaseChatMessageHistory] = {}

def get_session_history_1(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap the chain with message history
with_message_history_1 = RunnableWithMessageHistory(
    chain1,
    get_session_history_1,
    input_messages_key="messages",
)

# Function to chat with the bot
def chat_with_bot1(session_id: str, user_input: str):
    config = {"configurable": {"session_id": session_id}}
    response = with_message_history_1.invoke(
        {"messages": [{"content": user_input, "type": "human"}]},
        config=config
    )
    return response.content



This code creates a chat prompt template, sets up a chain combining the prompt and the Portkey-protected model, and establishes a system for managing chat history. The chat_with_bot function handles individual interactions, applying our guardrails to each message.


# Step 4: Running the chatbot
Finally, we'll implement the main loop for our chatbot. This will allow users to interact with the bot while our Portkey guardrails work in the background to protect sensitive information.

Try using this prompt to see guardrails in action- "My name is Siddharth and my email id is xyz@gmail.com"

In [ ]:
# Example usage
if __name__ == "__main__":
    session_id = "testWithGuradrails"

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

        response = chat_with_bot1(session_id, user_input)
        print(f"Bot: {response}")

## Conclusion

Here is how the traces of your request looks like on your Portkey dashboard-


<img src="https://raw.githubusercontent.com/siddharthsambharia-portkey/Portkey-Product-Images/main/PII-fail-traces-guardrails.png" alt="O" width="600" />

Portkey helps you connect to 250+ LLMs with just 2 lines of code. It helps you build robust aps with built in observability and guardrails.

In this tutorial, we've created two versions of a chatbot:

1. A basic chatbot without PII protection
2. An enhanced chatbot with Portkey guardrails


<img src="https://raw.githubusercontent.com/siddharthsambharia-portkey/Portkey-Product-Images/main/Portkey-Dashboard.png" alt="O" width="600" />



The first version demonstrates a simple implementation using LangChain, but it lacks any mechanism to detect or protect sensitive information. This could potentially lead to the exposure of PII in logs or responses.

The second version, integrating Portkey with our LangChain-based chatbot, significantly enhances our AI application's capabilities, reliability, and security. By implementing PII detection guardrails, we've added a crucial layer of protection against the unintended exposure of sensitive information.

By comparing these two implementations, we can clearly see the benefits of using Portkey guardrails in AI applications, especially when dealing with potentially sensitive user inputs.